In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [2]:
filters = {
    "item_conditions": {
        "New": 1000,
        "Open box": 1500,
        "Used": 3000,
        "Certified Refurbished": 2000,
        "Excellent - Refurbished": 2500,
        "Very Good": 3000,
        "Good": 4000,
        "For Parts or Not Working": 7000
    },
    "item_locations": {
        "Domestic": 1,
        "International": 2,
        "Continent": 3,
    },
    "directories": {
        "No Directory": 0,
        "Consumer Electronics": 9355,
        "Clothing, Shoes & Accessories": 11450,
        "Health & Beauty": 26395,
        "Home & Garden": 11700,
        "Sporting Goods": 382,
        "Toys & Hobbies": 220,
        "Books": 267,
        "Video Games & Consoles": 1249,
        "Collectibles": 1,
        "Business & Industrial": 12576,
        "Automotive": 6000, 
    },
    "categories": {
        "No Category": 0,
        "Cell Phones & Smartphones": 9355,
        "Laptops & Netbooks": 175673,
        "Watches": 31387,
        "Furniture": 3197,
        "Action Figures": 2605,
        "Jewelry & Watches": 281,
        "Cameras & Photo": 625,
        "Pet Supplies": 1281,
        "Crafts": 14339,
        "Computers/Tablets & Networking": 58058,
        "Cars & Trucks": 6001,  
        "Motorcycles": 6024,  
        "Car & Truck Parts": 6030,  
        "Motorcycle Parts": 10063,  
        "Automotive Tools & Supplies": 34998,  
    },
    "sort_order": {
        "Best Match": 12,
        "Time: ending soonest": 1,
        "Time: newly listed": 10,
        "Price + Shipping: lowest first": 15,
        "Price + Shipping: highest first": 16,
        "Distance: nearest first": 7
    }
}

In [3]:
# Define the base URL for the eBay search
url = "https://www.ebay.com/sch/i.html"

# Define the query parameters for the search request
params = {
    '_from': 'R40',
    '_nkw': 'iphone',
    'Brand': 'Apple',
    'LH_ItemCondition': filters["item_conditions"]["Used"],  # Item condition; 'New'.
    'LH_PrefLoc': filters["item_locations"]["Domestic"],
    '_udlo': '200',  # Minimum price.
    '_udhi': '400',  # Maximum price.
    '_dcat': filters["directories"]["No Directory"],  # Filter by directory ID; "Consumer Electronics".
    '_sacat': filters["categories"]["No Category"],  # Filter by category ID; "Cell Phones & Smartphones".
    '_sop': filters["sort_order"]["Time: newly listed"],  # Sort by "Time: newly listed"
    'LH_Sold': '0',  # Only sold listings (='1').
    #'LH_Complete': '1',  # Only completed listings (='1').
    'LH_BIN': '0',  # Only Buy It Now listings (='1').
    'LH_Auction': '1',  # Only Auction Listings (='1').
    'LH_BO': '0',  # Only listings that accept offers (='1').
    'LH_FS': '0',  # Only Free Shipping listings (='1').
    '_ipg': '240',  # Number of items per page (='1'), Max is 240.
    'rt': 'nc'  # Result type; 'nc' indicates no cache to ensure the search results are fresh. 
    
}

In [4]:
# Create a prepared request to inspect the full URL
request  =requests.Request('GET', url, params=params)
prepared_request = request.prepare()
print(prepared_request.url)

https://www.ebay.com/sch/i.html?_from=R40&_nkw=iphone&Brand=Apple&LH_ItemCondition=3000&LH_PrefLoc=1&_udlo=200&_udhi=400&_dcat=0&_sacat=0&_sop=10&LH_Sold=0&LH_BIN=0&LH_Auction=1&LH_BO=0&LH_FS=0&_ipg=240&rt=nc


In [5]:
# Initialize variables
page_number = 0
items_list = []

# Loop over pages
while True:
    
    page_number += 1
    
    print(f'Scraping page: {page_number}')
    request  =requests.Request('GET', url, params=params)
    prepared_request = request.prepare()
    print(prepared_request.url)
          
    params['_pgn'] = page_number
    
    # Send GET request to eBay with the defined parameters
    response  = requests.get(url, params=params)
    html_content = response.text # Get the HTML content of the page
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
          
    # Find the next button
    next_button = soup.find('button', class_='pagination__next', type='next')
  
    # Check if the next button is disabled
    if next_button and next_button.get('aria-disabled') == 'true':
        print('No more pages pages to scrape')
        break
    else:
        # Extract items
        items = soup.find_all('div', class_='s-item__wrapper clearfix')

        # Extract Listings
        for item in items [2:]:
            title = item.find('div', class_='s-item__title').text
            price = item.find('span', class_='s-item__price').text
            link = item.find('a', class_='s-item__link')['href'].split('?')[0]
            image_url = item.find('div', class_='s-item__image-wrapper image-treatment').find('img').get('src','No image URL')

            # Define each item as a dictionary
            item_dict = {
                'Title': title,
                'Price': price,
                'Link': link,
                'Image Link': image_url
            }

            # Append the dictionary to the list
            items_list.append(item_dict)

Scraping page: 1
https://www.ebay.com/sch/i.html?_from=R40&_nkw=iphone&Brand=Apple&LH_ItemCondition=3000&LH_PrefLoc=1&_udlo=200&_udhi=400&_dcat=0&_sacat=0&_sop=10&LH_Sold=0&LH_BIN=0&LH_Auction=1&LH_BO=0&LH_FS=0&_ipg=240&rt=nc
Scraping page: 2
https://www.ebay.com/sch/i.html?_from=R40&_nkw=iphone&Brand=Apple&LH_ItemCondition=3000&LH_PrefLoc=1&_udlo=200&_udhi=400&_dcat=0&_sacat=0&_sop=10&LH_Sold=0&LH_BIN=0&LH_Auction=1&LH_BO=0&LH_FS=0&_ipg=240&rt=nc&_pgn=1
Scraping page: 3
https://www.ebay.com/sch/i.html?_from=R40&_nkw=iphone&Brand=Apple&LH_ItemCondition=3000&LH_PrefLoc=1&_udlo=200&_udhi=400&_dcat=0&_sacat=0&_sop=10&LH_Sold=0&LH_BIN=0&LH_Auction=1&LH_BO=0&LH_FS=0&_ipg=240&rt=nc&_pgn=2
Scraping page: 4
https://www.ebay.com/sch/i.html?_from=R40&_nkw=iphone&Brand=Apple&LH_ItemCondition=3000&LH_PrefLoc=1&_udlo=200&_udhi=400&_dcat=0&_sacat=0&_sop=10&LH_Sold=0&LH_BIN=0&LH_Auction=1&LH_BO=0&LH_FS=0&_ipg=240&rt=nc&_pgn=3
No more pages pages to scrape


In [6]:
len(items_list)

720

In [8]:
items_df = pd.DataFrame(items_list)

In [21]:
items_df["is_new"] = items_df["Title"].str.contains('New').astype(int)
items_df["is_unlocked"] = items_df["Title"].str.contains('Unlocked').astype(int)
items_df["is_new_model"] = items_df["Title"].apply(str.lower).apply(lambda x: x[x.find('iphone') + 7:x.find('iphone') + 9]).apply(lambda x: 1 if x in ('13', '15') else 0)

In [22]:
items_df

,Title,Price,Link,Image Link,is_new,is_unlocked,is_old_model
0,New ListingApple iPhone 14 - 128 GB - Blue (Un...,$200.00,https://www.ebay.com/itm/306059902954,https://i.ebayimg.com/images/g/wGEAAOSw2H1nkoH...,1,1,0
1,New ListingiPhone 8 With TikTok Installed,$280.00,https://www.ebay.com/itm/396145539628,https://i.ebayimg.com/images/g/x70AAOSwvFdnkoC...,1,0,0
2,New ListingiPhone 15 Pro Max 1TB + TikTok app,$232.65,https://www.ebay.com/itm/387852657403,https://i.ebayimg.com/images/g/Hk4AAOSwO3Nnknu...,1,0,1
3,New ListingiPhone 12 64gb ( Unlocked With Tik...,$400.00,https://www.ebay.com/itm/176803670545,https://i.ebayimg.com/images/g/x~EAAOSw6qdnkoC...,1,1,0
4,New ListingiPhone 15 Pro Max TikTok Available,$250.00,https://www.ebay.com/itm/135513990760,https://i.ebayimg.com/images/g/Y6YAAOSwlzlnkoA...,1,0,1
...,...,...,...,...,...,...,...
715,Apple iPhone XS Max GOLD 64GB UNLOCKED Excelle...,$255.00,https://www.ebay.com/itm/235920291450,https://i.ebayimg.com/images/g/zCIAAOSwZ5pnfw4...,0,0,0
716,iPhone 13-midnight T-mobile Sim Locked,$200.00,https://www.ebay.com/itm/156643118560,https://i.ebayimg.com/images/g/L-QAAOSwj89nfvi...,0,0,1
717,New ListingIphone 12 pro,$250.00,https://www.ebay.com/itm/316152420993,https://i.ebayimg.com/images/g/v-wAAOSwD7hnfuW...,1,0,0
718,iPhone 11,$200.00,https://www.ebay.com/itm/316150223008,https://i.ebayimg.com/images/g/fBgAAOSwAKxnfb9...,0,0,0


In [11]:
items_df["Title"].apply(str.lower).apply(lambda x: x[x.find('iphone') + 7:x.find('iphone') + 9])
#a = "Apple iPhone 13 256GB Blue Locked To AT&T ….. Slightly Used ￼With 2 Phone Cases"
#start = a.find("iPhone") + 8
#a.substring(start,2)


0      14
1      8 
2      15
3      12
4      15
       ..
715    xs
716    13
717    12
718    11
719    13
Name: Title, Length: 720, dtype: object

In [24]:
items_df.to_csv("data/iphone_data.csv", encoding='utf-8', index=False)